## 优化数据处理流程

- 之前的数据处理流程为：
    - 根据给定的标签进行定向提取，并将数据分别保存在不同的基于小标签的文件中，
    - 再根据策划提供的无效属性值进行二次过滤


- 目前尝试的优化思路为：
    - 将第二阶段的过滤循环加入到第一阶段的过滤中

In [ ]:
label_list = []
import csv
import pandas as pd

# 人工选择的可用标签，并依据此标签进行词条分类
with open('./drive/MyDrive/Database/human_chosen_label/label/all_label.csv', 'r', encoding='utf-8-sig') as csvfile:
    csvReader = csv.DictReader(csvfile)
    for row in csvReader:
        label_list.append(row['label'])
print(label_list)

# 导入无效的属性值信息
useless_label_list = ['描述', '中文名', '中文名称', '外文名', '门票价格', '外文名称', '英文名', '英文名称','音标', '拼音', '注音', '简称', '拉丁名']

# 导入需要分类的词条数据信息
dataframe = pd.read_csv('./drive/MyDrive/Database/test_data/test_demo.csv', index_col=False)
dataframe.reset_index(drop=True, inplace=True)

['科学', '自然', '生物物种', '地理', '医学', '疾病', '科学百科疾病症状分类', '科学百科工程技术分类', '医学术语', '地形地貌', '科学百科生命科学分类', '科技术语', '科学百科信息科学分类', '科学百科数理科学分类', '科学百科环境生态分类', '自然现象', '科学百科天文学分类', '天体', '科学百科化学分类', '物理学', '山脉', '科学百科心理学分类', '科学百科地球科学分类', '天文', '自然资源', '科学百科航空航天分类', '科学百科其他分类', '地理学术语', '生物学', '数学', '细菌', '科学百科中医药分类', '中医', '药品', '西半球', '北美', '北半球', '科学百科药物分类', '自然地理', '语言', '字词', '生活', '地点', '动物', '植物', '宠物', '小吃', '菜品', '病毒', '传统节日', '鸟类', '木本植物', '烈士', '昆虫', '鱼类', '中国节日', '粤菜', '观赏植物', '鲁菜', '狗', '犬种', '饮品', '文化', '历史', '成语', '文化术语', '古代史', '历史人物', '二战', '历史事件', '文物考古', '非遗-非遗', '国家级-非遗', '外国历史', '近代史', '清朝', '遗址', '诗词', '民俗-非遗', '古生物', '二战人物', '神话人物', '传统戏剧-非遗', '外国历史事件', '中国历史事件', '历史年代', '传统体育、游艺与杂技-非遗', '古代历史事件', '世界级-非遗', '当代历史事件']


In [ ]:
print(dataframe[:5])

     实体   属性                                      值
0    胶饴   描述                        别名: 饴糖、畅糖、畅、软糖。
1    词条   描述  词条（拼音：cí tiáo）也叫词目，是辞书学用语，指收列的词语及其释文。
2    词条   标签                                     文化
3  红色食品   描述                红色食品是指食品为红色、橙红色或棕红色的食品。
4  红色食品  中文名                                   红色食品


In [ ]:
# 遍历词条数据，并进行分类任务
for i in range(len(dataframe)):
    i_label = dataframe.loc[i, '值']
    if i_label in label_list:
        print('i_label: ', i_label)
        filename = i_label + '.csv'
        save_path = './drive/MyDrive/Database/test_data/test_results/' + filename
        j = 1
        k = 0
        for j in range(i):
            if dataframe.loc[i, '实体'] == dataframe.loc[i-j, '实体']:
                j = j + 1
                if dataframe.loc[i-j+1, '属性'] == '标签':
                    k = k + 1
                else:
                    continue
            else:
                break
        # SettingwithCopyWarning -- Chained assignment
        each_data = dataframe.loc[i-j+1:i-k].copy()
        print('each_data: ', each_data)
        print('\n')
        # 删除无效的属性词条信息
        for useless_label in useless_label_list:
            each_data.drop(each_data.loc[each_data['属性']==useless_label].index, inplace=True) 
        print('each_data_delete_useless_label: ', each_data)
        print('\n')
        if each_data.empty:
            pass
        else:
            each_data.to_csv(save_path, mode='a', header=False, index=False, encoding='utf-8-sig') 

## 旧的基于标签的词条分类代码

- 缺点： 代码效率低，过多重复的循环，浪费时间

In [ ]:
# 遍历词条数据，并进行分类任务
for chosen_label in label_list:
    print('chosen_label: ', chosen_label)
    filename = chosen_label + '.csv'
    save_path = './drive/MyDrive/Database/new_test/results/' + filename
    for i in range(len(dataframe)):
        if dataframe.loc[i, '值'] == chosen_label:
            j = 1
            k = 0
            for j in range(i):
                if dataframe.loc[i, '实体'] == dataframe.loc[i-j, '实体']:
                    j = j + 1
                    if dataframe.loc[i-j+1, '属性'] == '标签':
                        k = k + 1
                    else:
                        continue
                else:
                    break
            each_data = dataframe.loc[i-j+1:i-k]
            print('each_data: ', each_data)
            print('\n')
            # 删除无效的属性词条信息
            for useless_label in useless_label_list:
                each_data.drop(each_data.loc[each_data['属性']==useless_label].index, inplace=True) 
            print('each_data_delete_useless_label: ', each_data)
            print('\n')
            if each_data.empty:
                pass
            else:
                each_data.to_csv(save_path, mode='a', header=True, index=False, encoding='utf-8-sig') 

## 测试DataFrame基于值进行行删除功能

- 测试demo

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.arange(20).reshape(5, 4),
                  columns=['A', 'B', 'C', 'D'])

data = df[:][:3]
print(df)
print(data)

In [ ]:
list_del = [3, 8]

In [ ]:
# 测试这种方法可行
for i in list_del[:]:
    data = data[data['A']!=i]
        # data.drop(data.loc[data['A']==i].index, inplace=True)

print(data)

   A  B  C  D
0  0  1  2  3
1  4  5  6  7


In [ ]:
df = pd.DataFrame(np.arange(20).reshape(5, 4),
                  columns=['A', 'B', 'C', 'D'])

data = df[:][:3]
list_del = [3, 8]

for i in list_del[:]:
    data.drop(data.loc[data['A']==i].index, inplace=True)

print(data)

   A  B  C  D
0  0  1  2  3
1  4  5  6  7


In [ ]:
data.drop(data.loc[data['A']==8].index, inplace=True)
print(data)

   A  B  C  D
0  0  1  2  3
1  4  5  6  7


In [ ]:
for i in range(6):
    print(i)

for j in range(5):
    print(j)

0
1
2
3
4
5
0
1
2
3
4


## 正则表达式测试

- 去除词条语料中的中括号中的内容

In [ ]:
#!/usr/bin/python
import re

line = "Learn Data, Python";

m = re.match( r'(\w+) (\w+)', line, re.M|re.I)

if m:
   print("m.group() : ", m.groups())
   print("m.group (1,2)", m.group(1, 2))
else:
   print("No match!!")

m.group() :  ('Learn', 'Data')
m.group (1,2) ('Learn', 'Data')


In [ ]:
test_line = '你好，我是个好人[其实是个坏人]'
m = re.findall(r'([..])', test_line)
print(m)

[]


In [ ]:
import re
x = '你好，我是个好人[其实是个坏人]，我想帮助你，不要给脸不要!'
re.sub("[\(\[].*?[\)\]]", "", x)

'你好，我是个好人，我想帮助你，不要给脸不要!'

In [ ]:
import re 
x = '你好，我是个好人[其实是个坏人]'
re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", x)

'你好，我是个好人[]'

In [ ]:
import re 
x = '你好，我是个好人[其实是个坏人]'
re.sub("([\(\[]).*?([\)\]])", "[]", x)

'你好，我是个好人[]'

In [ ]:
import re 
x = '你好，我是个好人(其实是个坏人)!'
re.sub("([\(]).*?([\)])", "", x)

'你好，我是个好人!'

In [ ]:
import re 
x = '你好，我是个好人(其实是个坏人)!'
re.findall("(.*?)",  x)

['',
 '你',
 '',
 '好',
 '',
 '，',
 '',
 '我',
 '',
 '是',
 '',
 '个',
 '',
 '好',
 '',
 '人',
 '',
 '(',
 '',
 '其',
 '',
 '实',
 '',
 '是',
 '',
 '个',
 '',
 '坏',
 '',
 '人',
 '',
 ')',
 '',
 '!',
 '']